In [1]:
#流程： DF  -> Gent_Vol生成WQ101 - > WINDSQL追加因子 -> Preprocess

In [6]:

import collections as col

from emulator.env_data import high2low, fix_data
from emulator.env_factor import get_factors

from Core.params import *


import re
import os
import numpy as np
import pandas as pd
import calendar
import matplotlib.pyplot as plt

from functools import reduce
from collections import defaultdict
from seaborn import heatmap
from copy import deepcopy
from scipy.optimize import linprog as lp
from datetime import datetime, timedelta, date
from collections import OrderedDict


#工作目录，存放代码
work_dir = os.path.dirname(os.path.abspath('__file__'))
#tickdata目录
tickdata_path = os.path.join(work_dir, 'tickdata')
#wq101 factors生成目录，用于合并
factors_path = os.path.join(work_dir, 'PQFactors')
#准备合成pct_chg_nm
PCT_CHG_path = os.path.join(work_dir, 'PCT_CHG_NM')
#最终合并目录
concat_path = os.path.join(work_dir, 'factor_preprocess', 'factors_rq')

def getting_factors(file_path):
    if os.path.isdir(file_path) == False:
        quotes = fix_data(os.path.join(tickdata_path, file_path))
        quotes = high2low(quotes, H2L_Freq)
    
        Index = quotes.index
        High = quotes.high.values
        Low = quotes.low.values
        Close = quotes.close.values
        Open = quotes.open.values
        Volume = quotes.volume.values
        
        factors = get_factors(Index, Open, Close, High, Low, Volume, rolling = 20, drop = False)
      
        code = file_path.split('.')[0] + '.' +  file_path.split('.')[1]
        
        
        if code.startswith('000300') is False or code.startswith('000905') is False:
            factors['code'] = code
            factors['industry_zx'] = quotes.industry_zx
            factors['name'] = quotes.name
            factors['PCT_CHG_NM'] = quotes.close.pct_change()
            factors.to_csv(os.path.join(factors_path, file_path), encoding='gbk')
        
        pct_chg = quotes.close.pct_change()
        pct_chg.to_csv(os.path.join(PCT_CHG_path, file_path), encoding='gbk')
        
    
def concat_date_factors(file_path):
    dates = [st_date, ed_date]
    start, end = [datetime.strptime(_, "%Y-%m-%d") for _ in dates]
    start = pd.date_range(start,  periods = 1, freq = 'BAS-JAN')[0].strftime("%Y-%m-%d")
    
    ds = pd.date_range(start,  periods = (end - start).days + 1, freq = 'd')
    tmp_ds_frame = pd.DataFrame(ds,columns = ['date'])
    dates_series = tmp_ds_frame.set_index(f.date).resample('W').bfill().index
    
    for _ in range(1,len(dates_series)):
        st = dates_series[_ - 1]
        ed = dates_series[_]
        
        factors = pd.DataFrame()
        for fpath in os.listdir(factors_path)[:]:
            tmp = pd.read_csv(os.path.join(factors_path, fpath), encoding="gbk", engine='c')
            tmp = tmp.set_index(tmp.tradeTime)
            factor_date_index = pd.to_datetime(tmp.tradeTime, format = '%Y-%m-%d')
            
            date_condition = (pd.to_datetime(tmp.tradeTime, format = '%Y-%m-%d') >= st) &\
                    (pd.to_datetime(tmp.tradeTime, format = '%Y-%m-%d') <= ed)
            
            if date_condition.any() == False:
                continue
                
            x = tmp[date_condition].iloc[-1,:]
            factors = pd.concat([factors, x], axis = 1)
                
        factors = factors.T
        try:
            factors = factors.set_index(factors.code)
        except:
            print(os.path.join(factors_path, fpath))
            print(factors.columns)
            print(_)
            
        factors.to_csv(os.path.join(concat_path, end_date)+'.csv', encoding='gbk')

def concat_chg_nm(file_path):
    pct_chg = pd.DataFrame()
    for fpath in os.listdir(PCT_CHG_path)[:]:
        #print(fpath)
        code = fpath.split('.')[0] + '.' +  fpath.split('.')[1]
        
        tmp = pd.read_csv(os.path.join(PCT_CHG_path, fpath), encoding="gbk", engine='c')
        tmp.rename(columns={'close': f'{code}'}, inplace=True)
        tmp = tmp.set_index(tmp.trading_point)
        pct_chg = pd.concat([pct_chg, tmp], axis = 1)
        del pct_chg['trading_point']
        
    pct_chg = pct_chg.T
    pct_chg.index.names =['code']
    print(pct_chg.columns)
    pct_chg.to_csv(os.path.join(work_dir, 'PCT_CHG_NM.csv'), encoding='gbk')




In [7]:
for fpath in os.listdir(tickdata_path):
    getting_factors(fpath)

KeyboardInterrupt: 

In [4]:
concat_date_factors(factors_path)

In [5]:
concat_chg_nm(PCT_CHG_path)

Index(['2012-07-31', '2012-08-31', '2012-09-30', '2012-10-31', '2012-11-30',
       '2012-12-31', '2013-01-31', '2013-02-28', '2013-03-31', '2013-04-30',
       ...
       '2020-02-29', '2020-03-31', '2020-04-30', '2020-05-31', '2012-01-31',
       '2012-02-29', '2012-03-31', '2012-04-30', '2012-05-31', '2012-06-30'],
      dtype='object', length=101)
